# 3D CAE Model Scapula Labelmap

In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
from loguru import logger
import numpy as np
import time
import pickle
import h5py
from skimage.transform import downscale_local_mean,resize
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score,calinski_harabasz_score,davies_bouldin_score
import seaborn as sns
from sklearn.cluster import KMeans,DBSCAN
import umap

import tensorflow as tf
import keras
from keras import backend as K
from keras.layers import Conv3D,Conv3DTranspose,Dense,MaxPooling3D,UpSampling3D,Flatten,Dense,Reshape,GlobalAveragePooling3D,BatchNormalization,Dropout
from keras.activations import relu
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from itkwidgets import view

# My imports (se si modifica il file bisogna riavviare tutto il kernel)
from utilities import plot_all_slices_notzero,plot_slices,dice_coef

## Import dataset

st=time.time()
with h5py.File("dataset_scapula_labelmap_res.hdf5","r") as f:
    scapula_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")

In [ ]:
st=time.time()
with h5py.File("dataset_scapula_labelmap_res_flipped.hdf5","r") as f:
    scapula_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")

In [ ]:
scapula_dataset.shape

## Downsampling

Dataset: 336x360x403\
Padding a: 336x360x408\
Downsampling per 4 a: 84x90x102

In [ ]:
scapula_dataset = np.pad(scapula_dataset,((0,0),(0,0),(0,0),(3,2)))

In [ ]:
scapula_dataset = resize(scapula_dataset,(scapula_dataset.shape[0],scapula_dataset.shape[1]/4,scapula_dataset.shape[2]/4,scapula_dataset.shape[3]/4),preserve_range=True,order=0,anti_aliasing=False)

In [ ]:
scapula_dataset.shape

In [ ]:
with h5py.File("resized_dataset_scapula_labelmap_res_flipped.hdf5","w") as f:
    f.create_dataset("mydataset",data=scapula_dataset)

# Model downsample

Una volta importato il dataset possiamo iniziare ad occuparci della costruzione del modello. Partiamo da immagini downscalate

st=time.time()
with h5py.File("resized_dataset_scapula_labelmap_res.hdf5","r") as f:
    scapula_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")

In [ ]:
st=time.time()
with h5py.File("resized_dataset_scapula_labelmap_res_flipped.hdf5","r") as f:
    scapula_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")

In [ ]:
scapula_dataset.shape

In [ ]:
resized_dataset_exp = np.expand_dims(scapula_dataset,axis=-1)
resized_dataset_exp.shape

In [ ]:
# Model downscaled con STRIDE (meglio)
embedding_dim = 128
inputs = keras.Input(shape=(resized_dataset_exp.shape[1],resized_dataset_exp.shape[2],resized_dataset_exp.shape[3],1))

# Encoder
x = Conv3D(filters=32,kernel_size=(3,3,3),padding="same",name="conv1_1")(inputs)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3D(filters=32,kernel_size=(3,3,3),strides=2,padding="same",name="conv1_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3D(filters=64,kernel_size=(3,4,4),padding="valid",name="conv2_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3D(filters=64,kernel_size=(3,3,3),strides=2,padding="same",name="conv2_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3D(filters=128,kernel_size=(3,4,3),padding="valid",name="conv3_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3D(filters=128,kernel_size=(3,3,3),strides=2,padding="same",name="conv3_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3D(filters=128,kernel_size=(3,3,3),strides=2,padding="valid",name="conv4")(x)
x = BatchNormalization()(x)
x = relu(x)

# Bottleneck
shape_before_encoding = x.shape[1:]
x = Flatten(name="flatten")(x) 
flattened_size = x.shape[1]
x = Dropout(0.3)(x)
embedding = Dense(embedding_dim,activation="relu",name="embedding")(x)
x = Dropout(0.3)(embedding)
x = Dense(flattened_size,activation="relu",name="expanding")(x)
x = Reshape(shape_before_encoding,name="reshape")(x)

# Decoder
x = Conv3DTranspose(filters=128,kernel_size=(3,3,3),strides=2,padding="valid",name="deconv4")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3DTranspose(filters=64,kernel_size=(3,3,3),padding="same",name="deconv3_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3DTranspose(filters=64,kernel_size=(3,3,3),strides=2,padding="same",name="deconv3_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3DTranspose(filters=32,kernel_size=(3,4,3),padding="valid",name="deconv2_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3DTranspose(filters=32,kernel_size=(3,3,3),strides=2,padding="same",name="deconv2_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3DTranspose(filters=16,kernel_size=(3,4,4),padding="valid",name="deconv1_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3DTranspose(filters=16,kernel_size=(3,3,3),strides=2,padding="same",name="deconv1_2")(x)
x = BatchNormalization()(x)
x = relu(x)

outputs = Conv3DTranspose(filters=1,kernel_size=(3,3,3),padding="same",name="deconv0",activation="sigmoid")(x)

model_downsample = keras.Model(inputs,outputs,name="model_downscaled")
model_downsample.summary()

In [ ]:
tot_loss = []
tot_val_loss = []
tot_dice = []
tot_val_dice = []

In [ ]:
with open("training_history/scap_flip_loss.pkl","rb") as f1,open("training_history/scap_flip_val_loss.pkl","rb") as f2,open("training_history/scap_flip_dice.pkl","rb") as f3,open("training_history/scap_flip_val_dice.pkl","rb") as f4:
    tot_loss = pickle.load(f1)
    tot_val_loss = pickle.load(f2)
    tot_dice = pickle.load(f3)
    tot_val_dice = pickle.load(f4)

In [ ]:
model_downsample.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[dice_coef]
)

checkpoint = ModelCheckpoint(
    "models_weights/scapula_flipped_checkpoint.weights.h5",
    monitor="loss",
    save_best_only=True,
    save_weights_only=True
)

history = model_downsample.fit(
    x=resized_dataset_exp,
    y=resized_dataset_exp,
    callbacks=[checkpoint],
    # validation_split=0.2,
    batch_size=16,
    epochs=50,
    verbose=1,
)

In [ ]:
model_downsample.save_weights("models_weights/scapula_flipped_loss0123_250ep_noval.weights.h5")

In [ ]:
tot_loss += history.history['loss'] # Per appendere liste tra loro
tot_val_loss += history.history['val_loss']
tot_dice += history.history['dice_coef']
tot_val_dice+=history.history['val_dice_coef']

In [ ]:
with open("training_history/scap_flip_loss.pkl","wb") as f1,open("training_history/scap_flip_val_loss.pkl","wb") as f2,open("training_history/scap_flip_dice.pkl","wb") as f3,open("training_history/scap_flip_val_dice.pkl","wb") as f4:
    pickle.dump(tot_loss,f1)
    pickle.dump(tot_val_loss,f2)
    pickle.dump(tot_dice,f3)
    pickle.dump(tot_val_dice,f4)

In [ ]:
# "Loss"
fig,ax = plt.subplots(2,1,figsize=(18,18))
start = 0
end = 200

# "Dice coefficient"
ax[0].plot(tot_loss[start:end+1],marker=".")
ax[0].plot(tot_val_loss[start:end+1],marker=".")
ax[0].set_title('Loss del Modello')
ax[0].set_ylabel('Loss')
ax[0].set_xlabel('Epoche')
ax[0].set_xticks([x for x in range(0,end+1-start,5)])
ax[0].set_xticklabels([x for x in range(start,end+1,5)])
ax[0].legend(['Train', 'Validation'], loc='upper left')
ax[0].grid()

# "Dice coefficient"
ax[1].plot(tot_dice[start:end+1],marker=".")
ax[1].plot(tot_val_dice[start:end+1],marker=".")
ax[1].set_title('Dice Coefficient del Modello')
ax[1].set_ylabel('Dice Coefficient')
ax[1].set_xlabel('Epoche')
ax[1].set_xticks([x for x in range(0,end+1-start,5)])
ax[1].set_xticklabels([x for x in range(start,end+1,5)])
ax[1].legend(['Train', 'Validation'], loc='upper left')
ax[1].grid()

## Predictions

In [ ]:
model_downsample.load_weights("models_weights/scapula_flipped_loss0123_250ep_noval.weights.h5")

In [ ]:
pred_ct = 200

In [ ]:
pred = model_downsample(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred_squeezed = np.squeeze(pred)
pred_squeezed_binary = np.where(pred_squeezed >= 0.5,1,0)

In [ ]:
pred_squeezed_binary.astype(np.uint8).dtype

In [ ]:
view(resized_dataset_exp[pred_ct]*255)

In [ ]:
view(pred_squeezed_binary.astype(np.uint8)*255)

In [ ]:
pred_squeezed_binary.dtype

In [ ]:
dice_coef(np.float32(resized_dataset_exp[pred_ct]),pred_squeezed)

In [ ]:
start_slice = 60
plot_slices(resized_dataset_exp[pred_ct],2,8,start_slice,title="Originale",titlesize=40)
plot_slices(pred_squeezed_binary,2,8,start_slice,title="Predizione Binaria",titlesize=40)
plot_slices(pred_squeezed,2,8,start_slice,title="Predizione",titlesize=40)

In [ ]:
# Carico i pesi 50 epoche
model_downsample.load_weights("models_weights/scapula_loss0210_50ep.weights.h5")
pred1 = model_downsample(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred1_squeezed = np.squeeze(pred1)
pred1_squeezed_binary = np.where(pred1_squeezed >= 0.5,1,0)
print(dice_coef(np.float32(resized_dataset_exp[pred_ct]),pred1_squeezed))

In [ ]:
# Carico i pesi 150 epoche
model_downsample.load_weights("models_weights/scapula_loss0163_100ep.weights.h5")
pred2 = model_downsample(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred2_squeezed = np.squeeze(pred2)
pred2_squeezed_binary = np.where(pred2_squeezed >= 0.5,1,0)
print(dice_coef(np.float32(resized_dataset_exp[pred_ct]),pred2_squeezed))

In [ ]:
# Carico i pesi 200 epoche
model_downsample.load_weights("models_weights/scapula_loss0143_150ep.weights.h5")
pred3 = model_downsample(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred3_squeezed = np.squeeze(pred3)
pred3_squeezed_binary = np.where(pred3_squeezed >= 0.5,1,0)
print(dice_coef(np.float32(resized_dataset_exp[pred_ct]),pred3_squeezed))

In [ ]:
# Carico i pesi 200 epoche
model_downsample.load_weights("models_weights/scapula_loss0129_200ep.weights.h5")
pred4 = model_downsample(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred4_squeezed = np.squeeze(pred4)
pred4_squeezed_binary = np.where(pred4_squeezed >= 0.5,1,0)
print(dice_coef(np.float32(resized_dataset_exp[pred_ct]),pred4_squeezed))

In [ ]:
start_slice = 64
plot_slices(resized_dataset_exp[pred_ct],1,8,start_slice,title="Originale",titlesize=40)
plot_slices(pred1_squeezed_binary,1,8,start_slice,title="Predizione dopo 50 epoche",titlesize=40)
plot_slices(pred2_squeezed_binary,1,8,start_slice,title="Predizione dopo 100 epoche",titlesize=40)
plot_slices(pred3_squeezed_binary,1,8,start_slice,title="Predizione dopo 150 epoche",titlesize=40)
plot_slices(pred3_squeezed_binary,1,8,start_slice,title="Predizione dopo 200 epoche",titlesize=40)

## Feature extraction

In [ ]:
model_downsample.load_weights("models_weights/scapula_flipped_checkpoint.weights.h5")

In [ ]:
feature_model = keras.Model(model_downsample.inputs,model_downsample.get_layer("embedding").output)

In [ ]:
features = feature_model.predict(resized_dataset_exp)

In [ ]:
features.shape

In [ ]:
with open(f"processing/scapula_res_flip_features_250ep.pkl","wb") as f:
    pickle.dump(features,f)

## Visualization (da qui in poi CPU)

In [ ]:
# UMAP
st = time.time()
features_umap = umap.UMAP(n_neighbors=30,random_state=42).fit_transform(features)
# n_neighbors basso, focus sulla local structure, alto rappresenta la struttura generale ma perde i dettagli. Range 2-100
print(f"Tempo dimensionality reduction UMAP: {time.time()-st:.2f} sec")

In [ ]:
print(features.shape)

In [ ]:
number_clusters = 10
kmeans = KMeans(n_clusters=number_clusters,n_init="auto",random_state=42)
labels = kmeans.fit_predict(features)

In [ ]:
# sns.set(rc = {'figure.figsize':(20, 15)})
plt.figure(figsize=(15,10))
sns.scatterplot(
    x = features_umap[:,0],
    y = features_umap[:,1],
    hue=labels,
    palette=sns.color_palette("bright",number_clusters)
)

In [ ]:
scapulae_indexes=[]
for i,lab in enumerate(labels):
    if lab == 3:
        scapulae_indexes.append(i)

In [ ]:
np.where(features_umap[:,0] <= 6)

In [ ]:
scapulae_indexes

## Clustering

In [ ]:
# Stampo l'inerzia (distanza within cluster) per vari numeri di cluster per vedere qual è il numero di cluster ottimale (gomito)
distortions = []
sil = []
dbs = []
chi = []

for k in range(1,10):
    kmeanModel = KMeans(n_clusters=k,n_init="auto")
    # kmeanModel.fit(features_pca)
    labels = kmeanModel.fit_predict(features_pca)
    distortions.append(kmeanModel.inertia_)
    if k > 1:
        sil.append(silhouette_score(features_pca,labels))
        dbs.append(davies_bouldin_score(features_pca,labels)) 
        chi.append(calinski_harabasz_score(features_pca,labels)) 

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))
sns.lineplot(x=range(2,10),y=sil,ax=ax[0],marker="o")
ax[0].set_title("Silhouette")  # Da -1 a 1, meglio 1
sns.lineplot(x=range(2,10),y=dbs,ax=ax[1],marker="o")
ax[1].set_title("Davies-Bouldin") # Più basso è meglio è
sns.lineplot(x=range(2,10),y=chi,ax=ax[2],marker="o")
ax[2].set_title("Calinski and Harabasz") # Più alto è meglio è

Tutti gli score sembrano indicare che il minor numero di cluster possibili, cioè 2, dia i risultati ottimali

In [ ]:
sns.lineplot(x=range(1,10),y=distortions,marker="o")
plt.title("Inertia")

Il gomito qui sembra intorno ai 3 cluster, come era emerso anche dal plot 2D

In [ ]:
# DBSCAN

dbscan = DBSCAN(eps=3)
labels = dbscan.fit_predict(features_tsne)
# Con questo divide bene i 3 cluster che si vedono a occhio
number_clusters = len(np.unique(labels))

In [ ]:
# KMEANS

number_clusters = 3
kmeans = KMeans(n_clusters=number_clusters,n_init="auto")
labels = kmeans.fit_predict(features_pca) # Sul tsne (2 dim) o sul pca (4 dim)? Meglio sul pca il Kmeans
centers = kmeans.cluster_centers_
centers

In [ ]:
data_scatter["labels"] = labels
fig, ax = plt.subplots(figsize=(15,10))
sns.scatterplot(
    data = data_scatter,
    x = "dim1",
    y = "dim2",
    hue= "labels",
    palette=sns.color_palette("bright",number_clusters),
    ax=ax
)
# Cluster centers, ha senso disegnarli solo se si fa il clustering sul tnse
#sns.scatterplot(x=centers[:,0],y=centers[:,1],marker="x",ax = ax,s=150,color="black")

I cluster che si sono formati e si vedono visualmente dividono scapola destra dalla sinistra. Quello in mezzo piccolo è formato da segmentazioni sbagliate che hanno poche slice. Con DBSCAN si riescono ad ottenere bene, con Kmeans no.

Nei 3 cluster che si formano con KMeans invece sembra che la differenza tra il primo e il terzo cluster, che contengono entrambi spalle destre, sia che il primo in generale contiene più slice.

In [ ]:
ct_labels = [[] for i in range(number_clusters)]
for i in range(len(labels)):
    ct_labels[labels[i]].append(i)
    #print(f"CT: {i+1}, label: {labels[i]}")

In [ ]:
print(f"Dimensioni dei cluster: {[len(el) for el in ct_labels]}")

In [ ]:
#ct_labels[0] # conterrà gli indici degli elementi del dataset appartenenti al primo cluster 

In [ ]:
plot_all_slices_notzero(resized_dataset[ct_labels[0][0]],title="Primo cluster") # Primo indice è il cluster, il secondo è la CT
plot_all_slices_notzero(resized_dataset[ct_labels[1][0]],title="Secondo cluster") # Primo indice è il cluster, il secondo è la CT
plot_all_slices_notzero(resized_dataset[ct_labels[2][0]],title="Terzo cluster") # Primo indice è il cluster, il secondo è la CT

## Heatmap

In [ ]:
# Provo ad ottenre una heatmap dell'ultimo livello dell'encoder per vedere dove si concentra l'estrazione delle feature

image_n = 110
conv_layer = model_downsample.get_layer("conv4")
heatmap_model = keras.Model(inputs=model_downsample.input, outputs=[conv_layer.output,model_downsample.output])
in_img = tf.Variable(np.expand_dims(resized_dataset_exp[image_n],axis=0),dtype=tf.float32)

with tf.GradientTape() as gtape:
    gtape.watch(in_img)
    heatmap_output, prediction = heatmap_model(in_img)
    
grads = gtape.gradient(prediction,heatmap_output)
chan_importance = tf.reduce_sum(grads,axis=(0,1,2,3))

heatmap = tf.reduce_mean(heatmap_output*chan_importance,axis=-1)

heatmap = tf.maximum(abs(heatmap),0)
heatmap /= tf.reduce_max(heatmap)

In [ ]:
heatmap.shape

In [ ]:
heatmap = np.squeeze(np.asarray(heatmap))
heatmap_res = resize(heatmap,output_shape=(in_img.shape[1],in_img.shape[2],in_img.shape[3]))
# heat_opencv = cv2.resize(heatmap,(in_img.shape[1],in_img.shape[2],in_img.shape[3]))

In [ ]:
ct_slice = 70

In [ ]:
heat_img = np.uint8(heatmap_res[:,:,ct_slice]*255)
in_img_cv = np.uint8(np.where(np.squeeze(in_img)[:,:,ct_slice] == 0,255,0)) # Cambio colori foreground e background per visualizzare meglio la heatmap 
heat_img = cv2.applyColorMap(heat_img, cv2.COLORMAP_JET)
in_img_cv = cv2.applyColorMap(in_img_cv, cv2.COLORMAP_BONE)
superimposed_img = heat_img*0.9 + in_img_cv*0.7
plt.imshow(np.clip(superimposed_img,0,255).astype("uint8"))

## Normal model

In [ ]:
#with tf.device("/GPU:0"):
inputs = keras.Input(shape=(scapula_dataset.shape[1],scapula_dataset.shape[2],scapula_dataset.shape[3],1))

# Encoder
x = Conv3D(filters=16,kernel_size=(4,4,3),activation="relu",padding="valid",name="conv1")(inputs)
x = MaxPooling3D(pool_size=(2,2,2),name="pool1")(x)

x = Conv3D(filters=32,kernel_size=(3,4,3),activation="relu",padding="valid",name="conv2")(x)
x = MaxPooling3D(pool_size=(2,2,2),name="pool2")(x)

x = Conv3D(filters=64,kernel_size=(3,3,4),activation="relu",padding="valid",name="conv3")(x)
x = MaxPooling3D(pool_size=(2,2,2),name="pool3")(x)

x = Conv3D(filters=128,kernel_size=(3,4,3),activation="relu",padding="valid",name="conv4")(x)
x = MaxPooling3D(pool_size=(2,2,2),name="pool4")(x)

shape_before_encoding = x.shape[1:]
print(shape_before_encoding)
# Feature extraction
x = Flatten(name="flatten")(x) 
flattened_size = x.shape[1]
print(flattened_size)
embedding = Dense(100,activation="relu",name="embedding")(x)
x = Dense(flattened_size,activation="relu",name="expanding")(embedding)
x = Reshape(shape_before_encoding,name="reshape")(x)

# Decoder
x = Conv3DTranspose(filters=64,kernel_size=(3,3,4),activation="relu",padding="same",name="deconv4")(x)
x = UpSampling3D((2,2,2),name="upsample4")(x)

x = Conv3DTranspose(filters=64,kernel_size=(3,4,3),activation="relu",padding="valid",name="deconv3")(x)
x = UpSampling3D((2,2,2),name="upsample3")(x)

x = Conv3DTranspose(filters=32,kernel_size=(3,3,4),activation="relu",padding="valid",name="deconv2")(x)
x = UpSampling3D((2,2,2),name="upsample2")(x)

x = Conv3DTranspose(filters=16,kernel_size=(3,4,3),padding="valid",name="deconv1")(x)
x = UpSampling3D((2,2,2),name="upsample1")(x)

outputs = Conv3DTranspose(filters=1,kernel_size=(4,4,3),padding="valid",name="deconv0",activation="sigmoid")(x)

model = keras.Model(inputs,outputs,name="model_1_200")
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.Accuracy()]
)

In [ ]:
print(scapula_dataset.shape)
in_shape = (scapula_dataset.shape[1],scapula_dataset.shape[2],scapula_dataset.shape[3],1)
scapula_dataset_expanded = np.expand_dims(scapula_dataset[:100],axis=-1) # solo i primi 100
scapula_dataset_expanded.shape

In [ ]:
#with tf.device("/GPU:0"):
model.fit(
    x=scapula_dataset_expanded,
    y=scapula_dataset_expanded,
    batch_size=2,
    epochs=2,
    verbose=1,
)

### Predictions

In [ ]:
pred_ct = 130

In [ ]:
pred = model_downsample.predict(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred_squeezed = np.squeeze(pred)
pred_squeezed_binary = np.where(pred_squeezed >= 0.5,1,0)

In [ ]:
np.min(pred)

In [ ]:
start_slice = 55
plot_slices(resized_dataset_exp[pred_ct],3,8,start_slice)
plot_slices(pred_squeezed_binary,3,8,start_slice)

In [ ]:
plot_all_slices_notzero(resized_dataset_exp[pred_ct])
plot_all_slices_notzero(pred_squeezed_binary)